In [2]:
import sys
import torch as torch
sys.path.append("/home/shaow/jupyter_base/pycharm_001")
base = "/home/shaow/jupyter_base/pycharm_001"
import graph_process as G
import keyword_objects as kw
import mysql_utils as mysql
import importlib
import obj.GraphBase as GB
import model.gcn as model
import numpy as np

In [22]:
# re-import kw for new feature
importlib.reload(mysql)
importlib.reload(kw)
importlib.reload(G)
importlib.reload(GB)
importlib.reload(model)

<module 'model.gcn' from '/home/shaow/jupyter_base/pycharm_001/model/gcn.py'>

In [3]:
appid_edge, appid_ros_edge, appid_app_edge, app_ros_edge, app_kw_edge, \
           id_index_map ,index_id_map, index_keyword_map, index_ros_map, index_app_map , \
           app_dict = G.process_year_graph(2019)

8.0.19
10
socket 10.0.82.237:3306
done init 3564 nodes from apply id
244800
done init 593205 nodes from application


In [4]:
gb_2019 = GB.GraphBase(appid_edge, appid_ros_edge, appid_app_edge, app_ros_edge, app_kw_edge,
           id_index_map, index_id_map, index_ros_map, index_app_map, index_keyword_map ,
           app_dict)

In [5]:
print(gb_2019.info())
print(len(app_kw_edge))

this scholar graph contains 
apply_code : 3565
ros : 12025
application : 233090
keyword : 344526

1085520


In [26]:
A_gb_2019 = gb_2019.get_split_appid_map('A', recursive=True)
B_gb_2019 = gb_2019.get_split_appid_map('B', recursive=True)
C_gb_2019 = gb_2019.get_split_appid_map('C', recursive=True)
D_gb_2019 = gb_2019.get_split_appid_map('D', recursive=True)
E_gb_2019 = gb_2019.get_split_appid_map('E', recursive=True)
F_gb_2019 = gb_2019.get_split_appid_map('F', recursive=True)
G_gb_2019 = gb_2019.get_split_appid_map('G', recursive=True)
H_gb_2019 = gb_2019.get_split_appid_map('H', recursive=True)

In [30]:
print(A_gb_2019.info())
print(B_gb_2019.info())
print(C_gb_2019.info())
print(D_gb_2019.info())
print(E_gb_2019.info())
print(F_gb_2019.info())
print(G_gb_2019.info())
print(H_gb_2019.info())
# print(np.array(A_gb_2019.are)[0])

this scholar graph contains 
apply_code : 321
ros : 1159
application : 16467
keyword : 35431

this scholar graph contains 
apply_code : 397
ros : 541
application : 15393
keyword : 26954

this scholar graph contains 
apply_code : 860
ros : 2356
application : 34706
keyword : 50082

this scholar graph contains 
apply_code : 168
ros : 1153
application : 18399
keyword : 41727

this scholar graph contains 
apply_code : 435
ros : 1880
application : 40272
keyword : 80639

this scholar graph contains 
apply_code : 796
ros : 2734
application : 24467
keyword : 51181

this scholar graph contains 
apply_code : 108
ros : 643
application : 12187
keyword : 25207

this scholar graph contains 
apply_code : 459
ros : 1704
application : 71199
keyword : 93168



In [32]:
gb_2019.id_index_map['E']
# base_gb , edge, label = A_gb_2019.negative_test_sampling_are()

1

In [29]:
gb_2019.id_index_map['E'] == gb_2019.id_index_map['root']
# print(edge[2], label[2])


False

In [9]:
device = 'cuda:0'
use_cuda = torch.cuda.is_available()
if use_cuda:
    torch.cuda.set_device(device)
model_state_file = 'model_state.pth'
forward_time = []
backward_time = []
lp_model = model.LinkPredict(A_gb_2019.node_count(),
                        256,
                        5,
                        rel_dict=A_gb_2019.rel_dict,
                        num_bases=128, #??? default = 100
                        num_hidden_layers=2,
                        dropout=0.2,
                        use_cuda=use_cuda,
                        reg_param=0.01)
optimizer = torch.optim.Adam(lp_model.parameters(), lr=0.01)
import time as time
print("start training...")
epoch = 0
best_mrr = 0
graph_base = A_gb_2019
grade_norm = 1.0
if use_cuda :
    lp_model.cuda()

start training...


In [13]:
print(A_gb_2019.node_count())

53378


In [11]:
while True:
    lp_model.train()
    epoch += 1

    # perform edge neighborhood sampling to generate training graph and data
    # g, node_id, edge_type, node_norm, data, labels = \
    #     utils.generate_sampled_graph_and_labels(
    #         train_data, args.graph_batch_size, args.graph_split_size,
    #         num_rels, adj_list, degrees, args.negative_sample,
    #         args.edge_sampler)
    # print("Done edge sampling")
    sam_gb ,neg_sample, label = graph_base.negative_test_sampling_are()
    g = sam_gb.get_whole_graph()
    g = g.to(torch.device('cuda:0'))
    # n_norm = sam_gb.comp_deg_norm()
    # g = sam_gb.node_norm_to_edge_norm(torch.from_numpy(n_norm).view(-1, 1))
    # set node/edge feature
    # node_id = torch.from_numpy(node_id).view(-1, 1).long()
    # edge_type = torch.from_numpy(edge_type)
    # edge_norm = model.node_norm_to_edge_norm(g, torch.from_numpy(node_norm).view(-1, 1))
    # data, labels = torch.from_numpy(data), torch.from_numpy(labels)
    # deg = g.in_degrees(range(g.number_of_nodes())).float().view(-1, 1)
    # if use_cuda:
    #     node_id, deg = node_id.cuda(), deg.cuda()
    #     edge_type, edge_norm = edge_type.cuda(), edge_norm.cuda()
    #     data, labels = data.cuda(), labels.cuda()

    t0 = time.time()
    embed = lp_model(g, torch.LongTensor(range(sam_gb.node_count())).reshape((-1, 1)).cuda())
    loss = lp_model.get_loss(g, embed, neg_sample, label)
    t1 = time.time()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(lp_model.parameters(), grade_norm) # clip gradients
    optimizer.step()
    t2 = time.time()

    forward_time.append(t1 - t0)
    backward_time.append(t2 - t1)
    print("Epoch {:04d} | Loss {:.4f} | Best MRR {:.4f} | Forward {:.4f}s | Backward {:.4f}s".
          format(epoch, loss.item(), best_mrr, forward_time[-1], backward_time[-1]))

    optimizer.zero_grad()

    # # validation
    # if epoch % 5 == 0:
    #     # perform validation on CPU because full graph is too large
    #     if use_cuda:
    #         model.cpu()
    #     model.eval()
    #     print("start eval")
    #     embed = model(test_graph, test_node_id, test_rel, test_norm)
    #     mrr = utils.calc_mrr(embed, model.w_relation, valid_data,
    #                          hits=[1, 3, 10], eval_bz=args.eval_batch_size)
    #     # save best model
    #     if mrr < best_mrr:
    #         if epoch >= args.n_epochs:
    #             break
    #     else:
    #         best_mrr = mrr
    #         torch.save({'state_dict': model.state_dict(), 'epoch': epoch},
    #                    model_state_file)
    #     if use_cuda:
    #         model.cuda()

print("training done")
print("Mean forward time: {:4f}s".format(np.mean(forward_time)))
print("Mean Backward time: {:4f}s".format(np.mean(backward_time)))

print("\nstart testing:")
# use best model checkpoint
checkpoint = torch.load(model_state_file)




RuntimeError: CUDA out of memory. Tried to allocate 2.01 GiB (GPU 0; 31.72 GiB total capacity; 29.46 GiB already allocated; 1.21 GiB free; 29.50 GiB reserved in total by PyTorch)

In [14]:
print(g)

Graph(num_nodes={'n': 53378},
      num_edges={('n', 'upon', 'n'): 320, ('n', 'contain', 'n'): 16467, ('n', 'a_include', 'n'): 0, ('n', 'b_include', 'n'): 8233, ('n', 'use', 'n'): 75516},
      metagraph=[('n', 'n'), ('n', 'n'), ('n', 'n'), ('n', 'n'), ('n', 'n')])
